# Data Modeling of Traffic Stops
> Jericho Timbol

## Problem Overview
For this problem we’ll use data released by the Stanford Open Policing Project (SOPP) for the state of North Carolina, available here https://stacks.stanford.edu/file/druid:py883nd2578/NC-clean.csv.gz. It contains records of 9.6 million police stops in the state between 2000 and 2015. Specifically we will be using this data to explore disparities in traffic stop metrics in regards to demographic groups.

In [1]:
#Importing Modules
import pandas as pd
import sklearn
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
import sklearn.preprocessing as preprocessing
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import numpy as np


In [2]:

#Loading and Observing Data first 1 million rows
tf_df = pd.read_csv("NC_cleaned.csv", low_memory = False, nrows= 1_000_000)
tf_df.head()

#Checking for unique values for each feature (viewed in seperate text editor)
for col in tf_df.columns:
    print(col)
    print(tf_df[col].unique())

id
['NC-2000-000001' 'NC-2000-000002' 'NC-2000-000003' ... 'NC-2001-435771'
 'NC-2001-435772' 'NC-2001-435773']
state
['NC']
stop_date
['2000-01-01' '2000-01-02' '2000-01-03' '2000-01-04' '2000-01-05'
 '2000-01-06' '2000-01-07' '2000-01-08' '2000-01-09' '2000-01-10'
 '2000-01-11' '2000-01-12' '2000-01-13' '2000-01-14' '2000-01-15'
 '2000-01-16' '2000-01-17' '2000-01-18' '2000-01-19' '2000-01-20'
 '2000-01-21' '2000-01-22' '2000-01-23' '2000-01-24' '2000-01-25'
 '2000-01-26' '2000-01-27' '2000-01-28' '2000-01-29' '2000-01-30'
 '2000-01-31' '2000-02-01' '2000-02-02' '2000-02-03' '2000-02-04'
 '2000-02-05' '2000-02-06' '2000-02-07' '2000-02-08' '2000-02-09'
 '2000-02-10' '2000-02-11' '2000-02-12' '2000-02-13' '2000-02-14'
 '2000-02-15' '2000-02-16' '2000-02-17' '2000-02-18' '2000-02-19'
 '2000-02-20' '2000-02-21' '2000-02-22' '2000-02-23' '2000-02-24'
 '2000-02-25' '2000-02-26' '2000-02-27' '2000-02-28' '2000-02-29'
 '2000-03-01' '2000-03-02' '2000-03-03' '2000-03-04' '2000-03-05'
 '2000-

In [3]:
#Getting More dataset information noteably for nulls
tf_df.info()

#Convert is_arrested and search_conducted
tf_df.replace({False: 0, True: 1}, inplace=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 27 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   id                     1000000 non-null  object 
 1   state                  1000000 non-null  object 
 2   stop_date              1000000 non-null  object 
 3   stop_time              335025 non-null   object 
 4   location_raw           667185 non-null   object 
 5   county_name            289288 non-null   object 
 6   county_fips            289288 non-null   float64
 7   fine_grained_location  338426 non-null   object 
 8   police_department      1000000 non-null  object 
 9   driver_gender          999997 non-null   object 
 10  driver_age_raw         999997 non-null   float64
 11  driver_age             999802 non-null   float64
 12  driver_race_raw        1000000 non-null  object 
 13  driver_race            999997 non-null   object 
 14  violation_raw      

### Part A

Report the percentage of the Arrested per each category, where percentage is calculated based on the total number of samples from that category.

> Race: Asian, Black, Hispanic, White

> Age: 15-19, 20-29, 30-39, 40-49, 50+

> Gender: female, male 


#### Race

In [4]:
#Race
print(pd.concat(
    [
        #Number of samples (value_counts() does not count na values)
        tf_df.groupby("driver_race")["is_arrested"].value_counts(),

        #Percentage of samples
        tf_df.groupby("driver_race")["is_arrested"].value_counts(normalize = True)
    ],
    keys=['counts', 'normalized_counts'],
    axis=1,
    ))

                         counts  normalized_counts
driver_race is_arrested                           
Asian       0              6492           0.984382
            1               103           0.015618
Black       0            218454           0.968990
            1              6991           0.031010
Hispanic    0             56337           0.929592
            1              4267           0.070408
Other       0             22378           0.968367
            1               731           0.031633
White       0            671254           0.981016
            1             12990           0.018984


#### Age

In [5]:
#Age

#Check max age for bins
print(tf_df.driver_age.max())

#Group by bins, include minimum value exclude top value. ( ]
print(
    pd.concat(
        [
tf_df.groupby(pd.cut(tf_df["driver_age"],
                    bins = [15,20,30,40,50,100], 
                    include_lowest= True, 
                    right = False))["is_arrested"].value_counts(),

tf_df.groupby(pd.cut(tf_df["driver_age"],
                    bins = [15,20,30,40,50,100], 
                    include_lowest= True, 
                    right = False))["is_arrested"].value_counts(normalize = True)],
                
                 keys=['counts', 'normalized_counts'],
                 axis=1,
                    
            ))

99.0
                        counts  normalized_counts
driver_age is_arrested                           
[15, 20)   0            111641           0.984315
           1              1779           0.015685
[20, 30)   0            358486           0.971638
           1             10464           0.028362
[30, 40)   0            237122           0.971851
           1              6868           0.028149
[40, 50)   0            149297           0.972936
           1              4153           0.027064
[50, 100)  0            118180           0.984899
           1              1812           0.015101


#### Gender

In [6]:
#Gender
print(pd.concat(
    [
        #Number of samples 
        tf_df.groupby("driver_gender")["is_arrested"].value_counts(),

        #Percentage of samples
        tf_df.groupby("driver_gender")["is_arrested"].value_counts(normalize = True)
    ],
    keys=['counts', 'normalized_counts'],
    axis=1,
    ))

                           counts  normalized_counts
driver_gender is_arrested                           
F             0            315028           0.988841
              1              3555           0.011159
M             0            659887           0.968408
              1             21527           0.031592


>Is the above metric a good indication of the biases that could be seen in the dataset?

No they aren't. This is because the distribution of the samples are widely uneven between the classes. Also the effects of different variables are different such as all 3 of the features we just analyzed (race, age and gender). In race for example, the first million records I sampled contains 6492 falling in the not arrested catagory. This is compared to the 671254 samples the class "White" has which is vastly different and affects representation and generalization considerations in the dataset.

### Part B

Controlling for age (bucketed as in part 1), gender, year use logistic regression to estimate impact of race on: search_conducted, is_arrested and stop_outcome == "Citation"

In [7]:
#Normalizing Feature Method
def data_transform(df):
    binary_data = pd.get_dummies(df)
    feature_cols = binary_data[binary_data.columns[:]]
    scaler = preprocessing.StandardScaler()
    data = pd.DataFrame(scaler.fit_transform(feature_cols), columns=feature_cols.columns)
    return data


In [8]:
#Subsetting dataframe Male, 20-29 Age, Year=2010 (Control Variables)
data_controlled = tf_df[(tf_df['driver_gender'] == "M") & (tf_df["stop_date"].str.contains("2000")) & (tf_df["driver_age"] >= 20) & (tf_df["driver_age"] <=29)]
data_controlled.info()
#Creating dataframe with race (target feature) to normalize 
data_controlled_race = data_controlled['driver_race']

#Using Normalizing method
data = data_transform(data_controlled_race)
data



<class 'pandas.core.frame.DataFrame'>
Int64Index: 142420 entries, 1 to 564226
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     142420 non-null  object 
 1   state                  142420 non-null  object 
 2   stop_date              142420 non-null  object 
 3   stop_time              84431 non-null   object 
 4   location_raw           58251 non-null   object 
 5   county_name            26630 non-null   object 
 6   county_fips            26630 non-null   float64
 7   fine_grained_location  85235 non-null   object 
 8   police_department      142420 non-null  object 
 9   driver_gender          142420 non-null  object 
 10  driver_age_raw         142420 non-null  float64
 11  driver_age             142420 non-null  float64
 12  driver_race_raw        142420 non-null  object 
 13  driver_race            142420 non-null  object 
 14  violation_raw          142420 non-nu

,Asian,Black,Hispanic,Other,White
0,-0.094288,-0.524665,-0.355931,-0.178045,0.762800
1,-0.094288,-0.524665,-0.355931,-0.178045,0.762800
2,-0.094288,1.905977,-0.355931,-0.178045,-1.310959
3,-0.094288,-0.524665,-0.355931,-0.178045,0.762800
4,-0.094288,1.905977,-0.355931,-0.178045,-1.310959
...,...,...,...,...,...
142415,-0.094288,-0.524665,2.809532,-0.178045,-1.310959
142416,-0.094288,-0.524665,-0.355931,-0.178045,0.762800
142417,-0.094288,-0.524665,-0.355931,-0.178045,0.762800
142418,-0.094288,1.905977,-0.355931,-0.178045,-1.310959


#### Search Conducted Probabilities

In [9]:
from sklearn.metrics import mean_squared_error
X = data
y = data_controlled.search_conducted

# Split X and y into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)

# Instantiate the model (using the default parameters)
logreg = LogisticRegression(random_state=16)

# Fit the model with data
logreg.fit(X_train, y_train)

#Find Probabilities
y_prob = logreg.predict_proba(X)
print ("Search Conducted Probabilities: \n" + str(y_prob))

Search Conducted Probabilities: 
[[0.98800193 0.01199807]
 [0.98800193 0.01199807]
 [0.97596989 0.02403011]
 ...
 [0.98800193 0.01199807]
 [0.97596989 0.02403011]
 [0.98800193 0.01199807]]


In [10]:
#Find Coefficients
w= logreg.coef_
print("Search Condcuted Coefficients (Asian, Black, Hispanic, Other, White) : \n" + str(w))

Search Condcuted Coefficients (Asian, Black, Hispanic, Other, White) : 
[[-0.12107505  0.10169805  0.24460957 -0.00534149 -0.22163781]]


In [11]:
#Standard Error Report
from sklearn.metrics import mean_squared_error
res=logreg.predict(X_train)
print("Search Conducted Standard Error is :" +  str(mean_squared_error(y_train,res)))

Search Conducted Standard Error is :0.017900107662781446


#### Arrested Probabilities

In [12]:
X = data
y = data_controlled.is_arrested

# Split X and y into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)

# Instantiate the model (using the default parameters)
logreg = LogisticRegression(random_state=16)

# Fit the model with data
logreg.fit(X_train, y_train)

#Find Probabilities
y_prob = logreg.predict_proba(X)
print ("Arrested Probabilities: \n" + str(y_prob))

Arrested Probabilities: 
[[0.97731201 0.02268799]
 [0.97731201 0.02268799]
 [0.96192157 0.03807843]
 ...
 [0.97731201 0.02268799]
 [0.96192157 0.03807843]
 [0.97731201 0.02268799]]


In [13]:
#Find Coefficients
w= logreg.coef_
print("Arrested Coefficients (Asian, Black, Hispanic, Other, White) : " + str(w))

Arrested Coefficients (Asian, Black, Hispanic, Other, White) : [[-0.1473407   0.04224526  0.29229119  0.02476736 -0.20783627]]


In [14]:
res=logreg.predict(X_train)
print("Arrested Standard Error is :" +  str(mean_squared_error(y_train,res)))

Arrested Standard Error is :0.03316949866591771


In [15]:
#### Stop Outcome is a citation Probabilities

In [16]:
#Needed to create and normalize new dataset since there are fewer records with "Citation" I need to convert
data_controlled_citation = data_controlled.copy()
data_controlled_citation.replace({'Arrest': 0,'Written Warning': 0, 'No Action' : 0,'Verbal Warning' :0,  'Citation': 1}, inplace=True)
data_citation_race = data_controlled_citation["driver_race"]


In [17]:
X = data
y = data_controlled_citation.stop_outcome

# Split X and y into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)

# Instantiate the model (using the default parameters)
logreg = LogisticRegression(random_state=16)

# Fit the model with data
logreg.fit(X_train, y_train)

#Probability of Citation
y_prob = logreg.predict_proba(X)
print ("Citation Recieved Probabilities: \n" + str(y_prob))

Citation Recieved Probabilities: 
[[0.19574498 0.80425502]
 [0.19574498 0.80425502]
 [0.21849071 0.78150929]
 ...
 [0.19574498 0.80425502]
 [0.21849071 0.78150929]
 [0.19574498 0.80425502]]


In [18]:
# Citation Coefficients
w= logreg.coef_
print("Citation Coefficients (Asian, Black, Hispanic, Other, White) : " + str(w))

Citation Coefficients (Asian, Black, Hispanic, Other, White) : [[ 0.02334619 -0.0303915  -0.02152667  0.01197036  0.03122316]]


In [19]:
#Standard Error
res=logreg.predict(X_train)
print("Citation Standard Error is :" +  str(mean_squared_error(y_train,res)))

Citation Standard Error is :0.20285540420352946


### Part C
Interpret the coefficients you reported in part B.

> What is the ratio of the probability of search of Hispanic drivers to White drivers? Black drivers to White drivers?

Hispanic-White : 
    The ratio of search was much higher in Hispanic versus White. The coefficient for the Hispanic class was .2446. For White, the b value was actually negative coming in at -.2216. 
    - Computing the white coefficient we can see that e^-.2216 = .801. 1 -.801 = .1988 which equates to about a 20% less relative chance to get searched at a traffic stop. 
    - Hispanics have an odd ratio of e^.2445 = 1.2771. If we subtract 1 we can see that hispanics are approximately 28% more likely to get searched at a traffic stop.

Black - White :
    -For Black with a much higher coefficient than white, e^.1017 = 1.1071. This equates to about a 11% higher chance to get searched.

> Repeat the above for the probability of arrest instead of search.

Hispanic - White:
-Hispanic has the higher coefficient. By calculating for odds ratio, e^.2923 = 1.3400 which is about a 34% higher chance to get arrested.
-White coefficient is once again negative at -.2078. e^-.2078 = .8124. By subracting this from 1 we get approx. 19% less chance to get arrested at a stop

Black - White:
-Black coeff is much higher than all groups at .4225. Calculations e^.4425 = 1.557. This amounts to a high percentage of a 56% relative chance to get arrested.

> What is the difference in citation probability between Hispanic drivers and White drivers? Black drivers and White drivers?

Across the three groups the coefficients are as follows:
 Black : -.0303 
 Hispanic: -.0215
 White: .0312

 The probability of citations are extremely close among the three groups being that the coefficients are close to one.On all three. They range from -.03 in the black group and .0312 in the white group. Probabilities do indicate that White has a very slight increased probability to get a citation. Hispanic and Black have a slightly lower chance of getting one.

### Part D

>Explain in a sentence or two why we should control for variables such as gender and location in the regression, and why the results might not be what we want if we don’t control for them. 

The aim of controlling variables in logistical regression is to eliminate other influencing factors or alternative explanations. In this case we are specifically seeing the independent variables (race) effect on the targets after including controlled variables. Analysis and prediction becomes more complex if we have to account for gender and race for example Black Male versus White Female in an arrest.

### Part E

>However, decisions about what to control are somewhat subjective. What is one reason we might not want to control for location in testing for discrimination? In other words, how might we underestimate discrimination if we control for location? 

If we control for location we underestimate discrimination by possibly zoning in on an area with skewed distributions of race within a population of citizens and the cops as well. Crime operations can also be rooted in certain geographic location furthering discrimination in policing in this area. Officers may be accustomed to higher stake situations and be on increased alert no matter the race during a traffic stop. Discrimination can be underestimated on various scales when talking about global levels. Counries can widely differ in policing systems and protocols.